In [1]:
import os
import csv
import sys
import random
import scipy
import numpy as np

from sets import Set


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'

/home/irashadow/env_TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: the sets module is deprecated


In [2]:
month_dict = dict()
month_dict['2015-01'] = 0
month_dict['2015-02'] = 1
month_dict['2015-03'] = 2
month_dict['2015-04'] = 3
month_dict['2015-05'] = 4
month_dict['2015-06'] = 5
month_dict['2015-07'] = 6
month_dict['2015-08'] = 7
month_dict['2015-09'] = 8
month_dict['2015-10'] = 9
month_dict['2015-11'] = 10
month_dict['2015-12'] = 11
month_dict['2016-01'] = 12
month_dict['2016-02'] = 13
month_dict['2016-03'] = 14
month_dict['2016-04'] = 15
month_dict['2016-05'] = 16
month_dict['2016-06'] = 17
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'


In [3]:
features_index = dict()
features_freq = dict()
feat_inx = 1

check_line_count = 0
feat_data = []
#for date_index_major in [9,11,12,13,15,16]:
for date_index_major in range(5,16):
    target_date = month_dict_inv[date_index_major]
    feat_data.append(open(workspace_path + '/L4_feat_standard/4_7_cust_date_nominal_prod_L4_'+str(target_date)+'_Model.feat', 'r'))
 
for feat_file in feat_data:
    for line in feat_file:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 7:
           print line 

        cust_id = line.split('|')[0]
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        added_lastmonth_product_features = line.split('|')[3]
        added_lastexist_product_features = line.split('|')[4]
        product_features = line.split('|')[5]
        nominal_features_pres = line.split('|')[6]
        real_features_pres = line.split('|')[7]
        

        if nominal_features_pres not in features_index:
           features_index[nominal_features_pres] = feat_inx
           features_freq[nominal_features_pres] = 1
           feat_inx += 1
        else:
           features_freq[nominal_features_pres] += 1 
        #----------------------------------------------------------------------------------    
                 
                                   
   
print check_line_count    
print feat_inx

406776
20163


In [5]:
updated_prod_feat = ['9254', '9235', '9233', '9433', '9218', '9238', '9442', '9454', '9022', '9216', '9223', '9224', '9317', '9351', '9418', '9409', '9438', '9449', '9453', '9435', '9517', '9536', '9618', '9408', '9219', '9237', '9240', '9242', '9253', '9311', '9243', '9507', '9403', '9405', '9331', '9239', '9249', '9420', '9422', '9443', '9452', '9531', '9315', '9203', '9717', '9718']

print len(updated_prod_feat)

46


In [8]:
import operator
from operator import itemgetter
features_index_selected = dict()
feat_inx = 1
sorted_x = sorted(features_freq.items(), key=operator.itemgetter(1), reverse = True)

for item in sorted_x[0:300]:
    
    nominal_features_selected = str(item[0])
        
    
    if nominal_features_selected not in features_index_selected:
       features_index_selected[nominal_features_selected] = dict() 
       for sub_item in updated_prod_feat: 
           features_index_selected[nominal_features_selected][sub_item] = feat_inx
           feat_inx += 1 


In [9]:
print len(features_index_selected)
print feat_inx

300
13801


In [10]:
check_line_count = 0
count_line1 = 0
count_line2 = 0 

feat_data = []
for date_index_major in range(5,17):
    feat_data_read = open(workspace_path + '/L4_feat_standard/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'r')
    feat_data_write = open(workspace_path + '/L4_feat_exp/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'w')
      

    for line in feat_data_read:
    
        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 6:
           print line 
        
        cust_id = int(line.split('|')[0])
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        added_lastmonth_product_features = line.split('|')[3]
        added_lastexist_product_features = line.split('|')[4]
        product_features = line.split('|')[5]
        nominal_features_pres = line.split('|')[6]
        real_features_pres = line.split('|')[7]
        realvalue_features_pres = line.split('|')[8]
    
        expand_features = np.zeros(feat_inx)    
        if nominal_features_pres in features_index_selected:
           count_line1 += 1 
           for realvalue_feat_item in realvalue_features_pres.split(' '):
                
               if realvalue_feat_item.split(':')[0] in features_index_selected[nominal_features_pres]:
                  get_index = features_index_selected[nominal_features_pres][realvalue_feat_item.split(':')[0]] 
                  expand_features[get_index] = float(realvalue_feat_item.split(':')[1]) 
                  
    
        expand_features_line = '' 
        for inx_item in list(np.where(expand_features > 0))[0]:
            expand_features_line += str(10000+inx_item) + ':%.2f '%float(expand_features[inx_item])

        prt_line = ''
        if len(expand_features_line) > 0:
           count_line2 += 1  
           prt_line = line + ' ' + expand_features_line[:-1] + '\n'   
        else:
           prt_line = line  + '\n' 

        feat_data_write.write(prt_line)
        
        
    feat_data_read.close()
    feat_data_write.close()

        
        
print check_line_count 
print count_line1
print count_line2

436493
237676
165886


In [12]:
check_line_count = 0
added_line_count = 0


feat_read = open(workspace_path + '/test/5_4_cust_date_prod_L4_standard.test', 'r')
feat_write = open(workspace_path + '/test/5_5_cust_date_prod_L4_expand.test', 'w')

for line in feat_read:
    
    line = line.strip()
    check_line_count += 1
    
    if check_line_count % 50000 == 0:
        
       print 'lines# after processed:' + str(check_line_count) 
    
    feat_set = line.split(' ')[1:]
    
                
    nominal_features_pres = ''  
    realvalue_features_pres = ''
    for feat_item in feat_set:
        if feat_item[0] == '5':
           nominal_features_pres += feat_item + ' '

        if feat_item[0] == '9':
           realvalue_features_pres += feat_item + ' '    
        
    expand_features = np.zeros(feat_inx)    
    if nominal_features_pres[:-1] in features_index_selected:
       added_line_count += 1 
       for realvalue_feat_item in realvalue_features_pres[:-1].split(' '):
           if realvalue_feat_item.split(':')[0] in features_index_selected[nominal_features_pres[:-1]]:
              get_index = features_index_selected[nominal_features_pres[:-1]][realvalue_feat_item.split(':')[0]] 
              expand_features[get_index] = float(realvalue_feat_item.split(':')[1])     
                        
    expand_features_line = '' 
    for inx_item in list(np.where(expand_features > 0))[0]:
        expand_features_line += str(10000+inx_item) + ':%.2f '%float(expand_features[inx_item])

    prt_line = ''
    if len(expand_features_line) > 0:
        
       prt_line = line + ' ' + expand_features_line[:-1] + '\n'   
    else:
       prt_line = line  + '\n' 

    feat_write.write(prt_line)
        
print check_line_count 
print added_line_count
feat_read.close() 
feat_write.close()


lines# after processed:50000
lines# after processed:100000
lines# after processed:150000
lines# after processed:200000
lines# after processed:250000
lines# after processed:300000
lines# after processed:350000
lines# after processed:400000
lines# after processed:450000
lines# after processed:500000
lines# after processed:550000
lines# after processed:600000
lines# after processed:650000
lines# after processed:700000
lines# after processed:750000
lines# after processed:800000
lines# after processed:850000
lines# after processed:900000
929615
193202
